This notebook iterates through the spreadsheet of images and calculates all of the radiomic data for each image.

In [ ]:
import os  # needed navigate the system to get the input data
import SimpleITK as sitk
import numpy as np
import radiomics
from radiomics import featureextractor  # This module is used for interaction with pyradiomics
# from radiomics import firstorder, getTestCase, glcm, glrlm, glszm, imageoperations, shape, shape2D
import six

In [ ]:
# First define the parameters
params = {}
params['shape2D'] = 1

# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor()
extractor.enableFeatureClassByName('shape2D')

print("Extraction parameters:\n\t", extractor.settings)
print("Enabled filters:\n\t", extractor.enabledImagetypes)
print("Enabled features:\n\t", extractor.enabledFeatures)

Iterate the spreadsheet and store the radiomics data

In [ ]:
from Common import Common
import matplotlib.pyplot as plt
import cv2
import pandas as pd
pd.set_option('display.max_rows', 200)

df = pd.read_csv (r'spreadsheets/BUSL_Notes.csv', skiprows=1)
df = df.dropna(axis="rows", how="all").dropna(axis="columns", how="all")

# Add in a column for filename, to match up the proper images
df["filename"] = df['Sample name'].str.split(expand=True)[0].str[2:].str.zfill(3)

Iterate the spreadsheet and calculate the radiomics data for each image

In [ ]:
all_data = []
img_dir = "images/"
extractor = featureextractor.RadiomicsFeatureExtractor()
extractor.enableFeatureClassByName('shape2D')
radiomics.setVerbosity(60)

for i, row in df.iterrows():
  filename = row["filename"]
  row_obj = { 'filename': filename, 'Histology_RLE': row["Histology_RLE"], 'Histology_Actual': row["Histology_Actual"], 'BI-RADS': row["BI-RADS"] }

  try:
    imagePath = img_dir + "Snap" + filename + "a.png"
    maskPath = img_dir + "Snap" + filename + "m.png"
    image = sitk.ReadImage(imagePath, sitk.sitkUInt16)
    mask = sitk.ReadImage(maskPath, sitk.sitkUInt16)
    mask = sitk.Clamp(mask,upperBound=1)  # Make sure the mask only has values of 0 or 1
    result = extractor.execute(image, mask)
    for key, value in result.items():
      if key.startswith('original'):
        row_obj[key] = value
  except:
    print("Error processing sample " + filename)
    continue
  
  all_data.append(row_obj)

new_df = pd.DataFrame(all_data)

In [ ]:
# new_df
new_df.to_csv('spreadsheets/radiomics.csv', index=False)